In [17]:
%load_ext autoreload
%autoreload 2

from final_project.loader import get_df
from final_project.models import log_reg
import final_project.builder as builder
from pathlib import Path
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
df = get_df(file='Flights_2018_1.csv')
df = builder.col
x = df.drop(columns=['ArrDel15'])
y = df[['ArrDel15']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=150)

In [19]:
lr_pipeline = log_reg.get_lr_pipeline()
param_grid = [
    log_reg.get_lr_param(solver="liblinear", penalty=["l1", "l2"]),
    log_reg.get_lr_param(solver="saga", penalty=["l2", "elasticnet"]),
]
grid_search = log_reg.get_lr_gridsearchcv(
    pipeline=lr_pipeline, param_grid=param_grid
)

In [21]:
logreg_model = grid_search.fit(x_train, y_train)

/home/drumblum/Northeastern/cs6140/CS6140-final-project/.venv/lib/python3.10/site-packages/sklearn/utils/multiclass.py:380: RuntimeWarning: invalid value encountered in cast
  if xp.any(data != data.astype(int)):


ValueError: Input y contains NaN.